In [176]:
import json

In [189]:
from datetime import datetime

In [177]:
import requests

In [178]:
import geopandas as gpd
import pandas as pd

In [ ]:
datetime.strptime

## Open 2023 Population Estimates

In [179]:
county_populations = pd.read_csv(
    "data/co-est2023-alldata.csv", sep=",", encoding="latin-1"
)

In [180]:
county_populations["FIPS"] = county_populations.apply(
    lambda row: f"{row['STATE']:02d}{row['COUNTY']:03d}", axis=1
)

In [181]:
county_populations_2023_df = county_populations[["POPESTIMATE2023", "FIPS"]].dropna()

## Get 1790 Populations

In [120]:
population_table_df = pd.read_csv("data/nhgis0013_ds1_1790_county.csv")
population_table_df = population_table_df[
    population_table_df["STATE"] == "South Carolina"
][["COUNTY", "AAQ001", "AAQ002", "AAQ003"]]
population_table_df["POPULATION_1790"] = (
    population_table_df.iloc[:, 1:4].sum(axis=1).astype(int)
)
population_table_df["NAME"] = population_table_df["COUNTY"]
population_table_df = population_table_df[["NAME", "POPULATION_1790"]]
population_table_df = population_table_df.replace(
    to_replace="Pendleton", value="Anderson"
)

## Open GIS Data

In [121]:
file_path = "data/cb_2018_us_county_500k/cb_2018_us_county_500k.shp"
counties_gdf = gpd.read_file(file_path)

## Merge Data

In [145]:
county_populations_2023_df["GEOID"] = county_populations_2023_df["FIPS"]
south_carolina_gdf = counties_gdf[counties_gdf["STATEFP"] == "45"][
    ["NAME", "geometry", "GEOID"]
]

In [146]:
south_carolina_current_population_gdf = south_carolina_gdf.merge(
    county_populations_2023_df, on="GEOID", how="left"
)

In [147]:
historic_sc_pop_gdf = south_carolina_current_population_gdf.merge(
    population_table_df, on="NAME", how="left"
)
historic_sc_pop_gdf = historic_sc_pop_gdf.fillna(0)

In [148]:
historic_sc_pop_gdf["change"] = (
    historic_sc_pop_gdf["POPESTIMATE2023"] - historic_sc_pop_gdf["POPULATION_1790"]
)

In [154]:
historic_sc_pop_gdf["change_percent"] = (
    (
        (
            historic_sc_pop_gdf["POPESTIMATE2023"]
            - historic_sc_pop_gdf["POPULATION_1790"]
        )
        / historic_sc_pop_gdf["POPESTIMATE2023"]
    ).round(decimals=3)
    * 100
).astype(float)

In [157]:
historic_sc_pop_gdf.sort_values("change_percent")

,NAME,geometry,GEOID,POPESTIMATE2023,FIPS,POPULATION_1790,change,change_percent
35,Edgefield,"POLYGON ((-82.18259 33.72662, -82.18275 33.729...",45037,27607,45037,13289.0,14318.0,51.9
13,Marlboro,"POLYGON ((-79.93432 34.80127, -79.93295 34.804...",45069,25704,45069,10706.0,14998.0,58.3
32,Abbeville,"POLYGON ((-82.74215 34.2119, -82.7401 34.21289...",45001,24434,45001,9197.0,15237.0,62.4
37,Fairfield,"POLYGON ((-81.42689 34.534, -81.42618 34.53677...",45039,20422,45039,7623.0,12799.0,62.7
3,Georgetown,"POLYGON ((-79.68088 33.3196, -79.67725 33.3367...",45043,65731,45043,22122.0,43609.0,66.3
19,Union,"POLYGON ((-81.854 34.59455, -81.84559 34.62364...",45087,26629,45087,7693.0,18936.0,71.1
10,Newberry,"POLYGON ((-81.94372 34.20605, -81.94196 34.208...",45071,38825,45071,9342.0,29483.0,75.9
43,Orangeburg,"POLYGON ((-81.37349 33.49051, -81.34131 33.517...",45075,82820,45075,18513.0,64307.0,77.6
38,Chester,"POLYGON ((-81.48822 34.73547, -81.48723 34.738...",45023,32226,45023,6866.0,25360.0,78.7
1,Charleston,"MULTIPOLYGON (((-79.50795 33.02008, -79.50713 ...",45019,424367,45019,66985.0,357382.0,84.2


In [150]:
historic_sc_pop_gdf.to_crs(3968).to_file("data/sc_populations_diff.gpkg")